In [39]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import time
import noisereduce
from IPython.display import Audio, display
import re
from num2tib.core import convert

In [40]:
class Timer:
    def __enter__(self):
        # Record start time
        self.start_time = time.time()
        return self  # Return self to optionally access elapsed time later
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # Record end time and calculate elapsed time
        self.end_time = time.time()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time:.4f} seconds")

In [41]:
model_id = "openpecha/mms-tts-sherab"

In [42]:
synthesiser_cpu = pipeline("text-to-speech", model_id)
synthesiser_gpu = pipeline("text-to-speech", model_id, device=0)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [65]:
def model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    model_size_mb = total_params * 4 / (1024 ** 2)
    print(f"Model size: {model_size_mb:.2f} MB")

In [66]:
model_size(synthesiser_cpu.model)

Model size: 138.43 MB


In [43]:
def replace_numbers_with_convert(sentence, wylie=True):
    pattern = r'\d+(\.\d+)?'
    def replace(match):
        return convert(match.group(), wylie)
        
    result = re.sub(pattern, replace, sentence)

    return result

def num2letter(sentence):
    tibetan_nums = "༠༡༢༣༤༥༦༧༨༩"
    for i, n in enumerate(tibetan_nums):
        sentence = sentence.replace(n, str(i))
    result = replace_numbers_with_convert(sentence, wylie=False)
    return result

In [44]:
examples = [
    "ད་དེ་ཚོ་འདི་བྱེད་དགོས་རེད་ ན་ཚ་ མ་ཡོང་སྔོན་ལ་ཁོ་རང་ལ་ཡང་ཁྱི་ཁོ་རང་ཁོ་ལ་ཡང་ཁབ་རྒྱག་ཡ་ཡོད་རེད། ཨུན་སྔོན་འགོག་དང་རཱབྷིསས་ཁབ་རྒྱག་ཡ་ཡོད་རེད་ད།",
    "སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།",
    "དངོས་གནས་ལབ་དགོས་རཱ་ད། མི་དབུལ་པོ་དེ་ཚོ་ལ་ག་རེ་ལབ་དགོས་རེད། སྦྱིན་པ་གཏང་ཡ་ཡོད་རཱ། ཨུན། དེ་འདྲ་གི་ལས་འགུལ་དེ་འདྲའི་མང་པོ་བརྩམས་ཀི་འདུག་བ། དེ་ཚོ་ཡང་ངས་ཚད་ལས་བརྒལ་བའི་ཡག་པོ་རེད་དྲན་གི་འདུག།",
    "ཁོང་རྣམ་པ་ནི་སྤྱིར་བཏང་གི་གང་ཟག་ཅིག་མ་ཡིན་པར་མི་རབས་ནས་མི་རབས་རྒྱུད་པ་འཛིན་པའི་ནོར་བུ་ཡིན་ཞིང་།",
    "ཨ་ལེ།  ཨེ་ནས་སྤྱིར་བཏང་ད་ང་ཚོ་ད་ལྟ་ཁྱེད་རང་གིས་དམིགས་ཡུལ་ད་གལ་ཆེན་པོ་བརྩིས་ནས།",
    "ཀུན་གླེང་གསར་འགྱུར། ༢༠༢༣་ལོའི་ཟླ་༩ ཚེས་༢༧ །",
]

In [59]:
def synthesis_speech(text, synthesiser):
    with Timer():
        text = num2letter(text)
        print(text)
        speech = synthesiser(text)
        audio = noisereduce.reduce_noise(y=speech["audio"], sr=speech["sampling_rate"])
        display(Audio(audio, rate=speech["sampling_rate"]))

In [61]:
for text in examples:
    synthesis_speech(text, synthesiser_cpu)

ད་དེ་ཚོ་འདི་བྱེད་དགོས་རེད་ ན་ཚ་ མ་ཡོང་སྔོན་ལ་ཁོ་རང་ལ་ཡང་ཁྱི་ཁོ་རང་ཁོ་ལ་ཡང་ཁབ་རྒྱག་ཡ་ཡོད་རེད། ཨུན་སྔོན་འགོག་དང་རཱབྷིསས་ཁབ་རྒྱག་ཡ་ཡོད་རེད་ད།


Elapsed time: 0.8595 seconds
སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།


Elapsed time: 1.6730 seconds
དངོས་གནས་ལབ་དགོས་རཱ་ད། མི་དབུལ་པོ་དེ་ཚོ་ལ་ག་རེ་ལབ་དགོས་རེད། སྦྱིན་པ་གཏང་ཡ་ཡོད་རཱ། ཨུན། དེ་འདྲ་གི་ལས་འགུལ་དེ་འདྲའི་མང་པོ་བརྩམས་ཀི་འདུག་བ། དེ་ཚོ་ཡང་ངས་ཚད་ལས་བརྒལ་བའི་ཡག་པོ་རེད་དྲན་གི་འདུག།


Elapsed time: 1.0998 seconds
ཁོང་རྣམ་པ་ནི་སྤྱིར་བཏང་གི་གང་ཟག་ཅིག་མ་ཡིན་པར་མི་རབས་ནས་མི་རབས་རྒྱུད་པ་འཛིན་པའི་ནོར་བུ་ཡིན་ཞིང་།


Elapsed time: 0.7646 seconds
ཨ་ལེ།  ཨེ་ནས་སྤྱིར་བཏང་ད་ང་ཚོ་ད་ལྟ་ཁྱེད་རང་གིས་དམིགས་ཡུལ་ད་གལ་ཆེན་པོ་བརྩིས་ནས།


Elapsed time: 0.5806 seconds
ཀུན་གླེང་གསར་འགྱུར། གཉིས་སྟོང་ཉི་ཤུ་རྩ་གསུམ་་ལོའི་ཟླ་དགུ་ ཚེས་ཉི་ཤུ་རྩ་བདུན །


Elapsed time: 0.6416 seconds
